In [4]:
#!/usr/bin/env python
# coding: utf-8

# # Глобальные переменные

# In[7]:


# Глобальные параметры, стандартные
list_not_data_for_view = []
loading_path_part = r'C:/Users/User/Desktop/py scripts/АИСОИП статусы kpi'
path_load_finish = r'C:/Users/User/Desktop/py scripts/АИСОИП отмены kpi'     # Где хранятся отмены и законченные производства.
path_crm = r"\\192.168.1.251\kpi"
run_prod = True



dict_log_pas = {"kpi":{'log':'901021350973_210540032049', 'pas':'Kk12345%'}
}


# In[8]:


count_row_in_load_excel = 10_000


# In[9]:


cnt_row_load_and_id = {100:0, 1000:1, 10_000:2, 20_000:3, 50_000:-1}
id_load_type = cnt_row_load_and_id[count_row_in_load_excel]


# ## Импорты

# In[10]:


from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from tqdm import tqdm
import time
import pandas as pd
from bs4 import BeautifulSoup
import os
import pickle
import math
from tqdm import tqdm

from datetime import date
from dateutil.relativedelta import relativedelta
from datetime import date
from datetime import datetime, timedelta
import numpy as np
from pathlib import Path

import sys
import traceback
from selenium.webdriver.support.wait import WebDriverWait
from dateutil.relativedelta import relativedelta
from selenium.webdriver.support import expected_conditions as EC
import os
import shutil
import math


def find_cnt_all_row(driver):
    '''
    example
    test_cnt = 1-100 из 4443
    cnt_all_row = 4443
    '''
    
#     class_find = '.v-data-iterator.excel-footer .v-data-footer__pagination'
    css_el = '.v-data-iterator.mt-4.mt-md-9 .v-data-footer__pagination'
    test_cnt = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, css_el))
    ).text
    time.sleep(1)
    
#     class_find = '.v-data-iterator.mt-4.mt-md-9 .v-data-footer__pagination'
#     test_cnt = driver.find_element(By.CSS_SELECTOR, class_find).text
    cnt_all_row = int(test_cnt.split(' из ')[-1])
    cnt_iter = math.ceil(cnt_all_row / count_row_in_load_excel)
    print(test_cnt)
    return cnt_iter, test_cnt




def click_next_excel_50k(driver):
    css_el = '.v-data-iterator.excel-footer .v-data-footer .v-data-footer__icons-after .v-icon.notranslate.material-icons.theme--light'
    field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, css_el))
    )
    field.click()
    time.sleep(10)
    

# pd.set_option('max_columns', None)
def click_load1(driver):
    css_el = '.d-flex.justify-end .v-btn.v-btn--icon.v-btn--round.theme--light.v-size--default .v-btn__content'
    field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, css_el))
    )
    field.click()
    time.sleep(2)
    

def load_need_value(driver):
    """
    Выбираю по сколько дел буду скачивать.
    """
    # Перехожу на нужную вкладку

    css_el = '.v-input.v-input--hide-details.v-input--is-label-active.v-input--is-dirty.theme--light.v-text-field.v-text-field--is-booted.v-select .v-icon.notranslate.material-icons.theme--light'
    field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, css_el))
    )
    field.click()
    
    
    css_el = '.v-list.v-select-list.v-sheet.theme--light.theme--light .v-list-item__content'
    field = driver.find_elements(By.CSS_SELECTOR, css_el)[id_load_type]
    field.click()
    time.sleep(2)

    
    

def click_load2(driver):
    css_el = '.v-btn.v-btn--is-elevated.v-btn--has-bg.theme--light.v-size--small.primary .v-btn__content'
    field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, css_el))
    )
    field.click()
    time.sleep(100)

def date_to_str(date_):
    day_   = str(date_.day)
    month_ = str(date_.month)
    year_ = str(date_.year)
    if len(day_) == 1:
        day_ = '0' + day_
    if len(month_) == 1:
        month_ = '0' + month_
    return f'{day_}.{month_}.{year_}'

def to_site(loading_path_all):

    # Вхожу на сайт
    url = 'https://aisoip.adilet.gov.kz/cabinet/exec-productions'

    # Добавляю папку сохранения 
    #https://stackoverflow.com/questions/34515328/how-to-set-default-download-directory-in-selenium-chrome-capabilities
    #https://stackoverflow.com/questions/18026391/selenium-webdriver-in-python-files-download-directory-change-in-chrome-prefere
    
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    prefs = {"profile.default_content_settings.popups": 0,
             "download.default_directory": (loading_path_all + '/').replace('/', '\\'), # IMPORTANT - ENDING SLASH V IMPORTANT
             "directory_upgrade": True,
             "profile.managed_default_content_settings.images": 2}
    options.add_experimental_option("prefs", prefs)
    
    
    # Подключение к сайту
#     driver = webdriver.Chrome(chrome_options=options)
    driver = webdriver.Chrome(options=options)
        
    
    driver.implicitly_wait(10)
    driver.get(url)

    time.sleep(2)
    
    return driver
    
    
def write_login(driver, name_login):
    '''Ввожу логин пароль'''
    login_global = dict_log_pas[name_login]['log']
    pasword_global = dict_log_pas[name_login]['pas']    
    

    login  = login_global
    pasword = pasword_global
    
    time.sleep(0.5)

    
    field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[name="usernameUserInput"]'))
    )

    field.send_keys(login)

    
    field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "password"))
    )

    field.send_keys(pasword)
    
    time.sleep(0.5)
    
    css_el = '.buttons>.ui.primary.button.fluid'
    field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, css_el))
    )
        
    field.click()
    
    time.sleep(0.5)
    
    
def to_targer_sheet_1(driver):
    # Перехожу на нужную вкладку
    # исполнительные производства
    css_el = '.cabinet-header>.navigation>:nth-child(2)'
    field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, css_el))
    )
    field.click()

       
    
def to_targer_sheet_2(driver):
    # Перехожу на нужную вкладку

    css_el = '.v-slide-group__content.v-tabs-bar__content>:nth-child(3)'
    field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, css_el))
    )
    field.click()
    
    
def set_100_row(driver):
    time.sleep(0.5)
    field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.v-input.v-input--hide-details.v-input--is-label-active.v-input--is-dirty.theme--light.v-text-field.v-text-field--is-booted.v-select .v-select__selection.v-select__selection--comma'))
    )

    field.click()
    
    
    field = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.v-list.v-select-list.v-sheet.theme--light.theme--light>:nth-child(4)'))
    )

    field.click()
    
    
def wait_load_web(driver, max_sec_wait=30):
    driver.implicitly_wait(0.1)
    t_start = time.time()
    while True:
        time.sleep(0.1)
        t_now = time.time()
        
        if t_now - t_start >= max_sec_wait:
            print('Слишком долгая загрузка сайта. Обновляю странцу.')
            driver.refresh()
            break
        
        try:
            driver.find_element(By.CSS_SELECTOR, '.v-progress-linear__buffer')
            continue
        except:
            break
        
    driver.implicitly_wait(10)
    

#############
def make_load_dir(name_login, loading_path_part):
    '''Создаю папки в которые сохраняю все данные'''
    
    now = datetime.now()
    dt_string = now.strftime("%Y-%m-%d___%H-%M")
    print("date and time =", dt_string)

    # Создаю папку храенния
    loading_path = f'{loading_path_part}/{name_login} {dt_string}'
    #loading_path_d = f'{loading_path}/day_agg' 
    loading_path_all = f'{loading_path}/all_file'
    dowlend_path = loading_path_all

    os.mkdir(loading_path)
    # os.mkdir(loading_path_d)
    os.mkdir(loading_path_all)
    loading_path_d = loading_path_all
    
    return loading_path, loading_path_d, loading_path_all, dowlend_path

# 2021, 1, 10

def parsing_all(name_login, date_start, loading_path, loading_path_d, loading_path_all, dowlend_path, list_not_data_for_view):
    '''Основной скрипт для парсинга'''
    
    if date_start is None: #'IDC':date(2021, 1, 18)
        dict_date_start = {'kpi':date(2021, 1, 1)}
        date_start = dict_date_start[name_login]


    date_finish = date.today()
    date_end = date_finish
    break_flag = False

    status_pars = {'new_driver':True, 'new_log':True, 'set_web_list_targ':True, 'set_iin':True, 'set_100':True}
    cnt_load_file_i = 0
    try_while = 0
    for i in range(1, 200):
        # Параметры по умолчанию
        error_list = []
        try_while += 1
        print('try = ', i)
        
        if break_flag:
            break

        
        
        try:
            t_start_sec = time.time()
        
            
            if status_pars['new_driver']:
                driver = to_site(loading_path_all)
                status_pars['new_driver'] = False
                
            if len(driver.window_handles) > 1:
                print('Добавлена новая вкладка. Это условие остановки цикла.')
                break_flag = True
                break
                

            if status_pars['new_log']:# or login_new:
                write_login(driver, name_login)
                status_pars['new_log'] = False
                
            if status_pars['set_web_list_targ']:
                to_targer_sheet_1(driver)
                to_targer_sheet_2(driver)
                status_pars['set_web_list_targ'] = False


#             if date_end >= date_finish:
#                 break

            ## Цыкл по датам.
#             try_while = 0
             #while date_end <= date_finish:
                # try_while += 1
                
            if len(driver.window_handles) > 1:
                print('Добавлена новая вкладка. Это условие остановки цикла.')
                break_flag = True
                break
            
            # Ишу файлы скаченные до этого
            file_load_before = []
            for f_name in os.listdir(dowlend_path):
                dowlend_file_p = dowlend_path + '//' + f_name
                if 'Взыскатель-' in dowlend_file_p and '.xlsx' in dowlend_file_p:
                    file_load_before.append(dowlend_file_p)

            # date_start = date_end
            print(date_end)
            date_start_srt = date_to_str(date_start)
            date_end_srt   = date_to_str(date_end)

            # Установить дату старта
            butom_date_clear = 'v-input__append-inner'
            field = driver.find_elements(By.CLASS_NAME, butom_date_clear)[0].click()

            butom_date = 'v-text-field__slot'
            field = driver.find_elements(By.CLASS_NAME, butom_date)[0]
            field = field.find_element(By.CSS_SELECTOR, 'input')

            field.send_keys(date_start_srt)
            time.sleep(1)

            # Установка даты окончания 
            butom_date_clear = 'v-input__append-inner'
            field = driver.find_elements(By.CLASS_NAME, butom_date_clear)[1].click()

            butom_date = 'v-text-field__slot'
            field = driver.find_elements(By.CLASS_NAME, butom_date)[1]
            field = field.find_element(By.CSS_SELECTOR, 'input')

            field.send_keys(date_end_srt)
            time.sleep(0.5)    

            # Нажать на поиск
            butom_start_find = '.mb-2.mr-2.mr-md-5.v-btn.v-btn--is-elevated.v-btn--has-bg.theme--light.v-size--default.primary'
            field = driver.find_element(By.CSS_SELECTOR, butom_start_find).click()
            time.sleep(0.5)
            wait_load_web(driver) # Ожидание если грузит.
            time.sleep(0.5)

                # text_data = ''

                # # Сохраняю Количество данных "из"
                # class_find = 'v-data-footer__pagination'.replace(' ', '.')
                # count_line = driver.find_element(By.CLASS_NAME, class_find).text

                # list_not_data_for_view.append([date_end_srt, count_line])   

                # if 'Нет данных для отображения' in text_data or count_line == '–':

                #     time.sleep(0.5)
                #     date_end = date_end + timedelta(days=1)
                #     continue

#             if status_pars['set_100']:
#                 set_100_row(driver)

            cnt_iter, cnt_row_text = find_cnt_all_row(driver)
            status_pars['set_100'] = False
            #new load
            click_load1(driver)
            load_need_value(driver)
            
            
            for i_iter_list in range(cnt_iter):
                files_before = set(os.listdir(loading_path_all))
                
                
                if i_iter_list < cnt_load_file_i:
                    click_next_excel_50k(driver)
                    continue
                
                click_load2(driver)
                files_current = set(os.listdir(loading_path_all))
                new_files = files_current - files_before
                if new_files:
                    cnt_load_file_i += 1
                
                if i_iter_list >= (cnt_iter - 1):
                    continue
                click_next_excel_50k(driver)
                try_while = 0
            
            list_not_data_for_view.append([date_end_srt, cnt_row_text])
            if cnt_load_file_i > i_iter_list:
                break

        except Exception as e:
            
            exc_type, exc_value, exc_traceback = sys.exc_info()
            tb = traceback.extract_tb(exc_traceback)
            str_error = e
            if tb:
                # The first item in the traceback list is the location of the error
                file, line_number, function, text = tb[0]

                str_error = f"{e}\n Error occurred in file {file}, line {line_number}: {text}"
            print(str_error)
            error_list.append(str_error)


            if try_while <= 2:
                driver.refresh()
                status_pars = {'new_driver':False, 'new_log':True, 'set_web_list_targ':True, 'set_iin':True, 'set_100':True}
                time.sleep(5)
            else:
                driver.refresh()
                status_pars = {'new_driver':True, 'new_log':True, 'set_web_list_targ':True, 'set_iin':True, 'set_100':True}
                time.sleep(5)
                try_while = 0
                
                try:
                    driver.quit()
                except:
                    pass



            continue


# In[11]:


def save_all_date(loading_path, list_not_data_for_view, loading_path_d):
    '''Сохраняю историю парсинга. Сохраняю Объедененный файл'''
        
    now_str = datetime.now().strftime("%d.%m.%Y")
    pd.DataFrame(list_not_data_for_view).to_excel(f'{loading_path}/Были ли данные {now_str}.xlsx')
    
    
    list_agg_day_file = []
    for f_name in os.listdir(loading_path_d):
        if f_name[0] == '~':
            continue

        dowlend_file_p = loading_path_d + '//' + f_name
        list_agg_day_file.append(dowlend_file_p)
        
    print()    
    df_concat = pd.DataFrame()
    for f in tqdm(list_agg_day_file):
        df = pd.read_excel(f, dtype={'ИИН/БИН должника':str})
        df_concat = pd.concat([df_concat, df])

    print('Количество строк в объедененном файле =', df_concat.shape[0])    
    
    if df_concat.shape[0] == 0:
        print('В данном АИСОИП, по данным датам нет данных. Переходим к следующему.')
        return None
    print('Удаление дублей, Правка №')
    df_final = df_concat.reset_index(False).drop(['index', '№'], axis=1)
    df_final = df_final.drop_duplicates()
    df_final = df_final.reset_index(drop=True).reset_index().rename(columns={'index':'№'})
    df_final['№'] = df_final['№'] + 1
    
    
    now_str = datetime.now().strftime("%d.%m.%Y")
    df_final.to_excel(f'{loading_path}/{name_login} AIS_OIP {now_str}.xlsx', index=False)
    
    print('Количество строк в объедененном файле без дублей =', df_final.shape[0])
    return df_final

def cleaning_date(df_final):
    '''Очищаю номер года '''
    
    year_recode = {'1202':'2021', 
                   '0202':'2020',
                   '0222':'2022',
                   '0223':'2023',
                   '1120':'2021',
                   '0320':'2023',
                   '0221':'2021',
                   '0218':'2018'
                  }

    def to_date(str_d):
        try:
            split_srt = str_d.split('.')
            if len(split_srt) < 3:
                print(f'Дата {str_d} не может быть преобразована')
                return np.nan

            year_ =  split_srt[-1]
            if '199' == year_:
                print(f'Зануляю дату {str_d}')
                return np.nan

            for year_false, year_true in year_recode.items():
                if year_false == year_:
                    print(f'Заменяю год в дате {str_d} : с {year_true} на {year_false}')
                    return pd.to_datetime(str_d.replace(year_false, year_true), format="%d.%m.%Y")

            return pd.to_datetime(str_d, format="%d.%m.%Y")

        except:
#             assert False, f'Ошибка, строка = "{str_d}" не конвертируется'
            print(f'Ошибка, строка = "{str_d}" не конвертируется')
            print('Подставлен NaN')
            return np.nan


    df_final = df_final.copy()


    for n_date in ['Дата возбуждения', 'Дата выписки исполнительного документа']:
        print('   ', n_date)
        df_final[n_date] = df_final[n_date].apply(to_date)
        print('+ Все даты верные')

    #df_final.drop('date_end', axis=1, inplace=True)
    
    return df_final



 


# # Парсинг

# In[12]:


for name_login in dict_log_pas.keys():
    print('***', name_login, '***')


    date_start = None
    list_not_data_for_view = []
    
    loading_path, loading_path_d, loading_path_all, dowlend_path = make_load_dir(name_login, loading_path_part)
    
    parsing_all(name_login, date_start, loading_path, loading_path_d, loading_path_all, dowlend_path, list_not_data_for_view)

    df_final = save_all_date(loading_path, list_not_data_for_view, loading_path_d)
    
    if df_final is None:
        continue
        
    df_final = cleaning_date(df_final)
    # if name_login == 'IDC':
    #     df_final_idc = idc_choice_row_iin(df_final, loading_path, name_login)
    # else:
    #     other_choice_row_iin(df_final, loading_path, name_login, df_final_idc)
        
        
    try: 
        time.sleep(5)
        driver.close()
    except:
        pass
    try: 
        time.sleep(2)
        driver.quit()
    except:
        pass
    driver = None


# In[13]:


# 1/0


# # Сохранение итогов

# In[15]:


df_final


# ## Объедененный файл с правками

# In[ ]:


# year_recode = {'1202':'2021', 
#                '0202':'2020',
#                '0222':'2022',
#                '0223':'2023',
#                '1120':'2021',
#                '0320':'2023',
#                '0221':'2021',
#                '0218':'2018'
#               }

# def to_date(str_d):
#     try:
#         split_srt = str_d.split('.')
#         if len(split_srt) < 3:
#             print(f'Дата {str_d} не может быть преобразована')
#             return np.nan
        
#         year_ =  split_srt[-1]
#         if '199' == year_:
#             print(f'Зануляю дату {str_d}')
#             return np.nan
        
#         for year_false, year_true in year_recode.items():
#             if year_false == year_:
#                 print(f'Заменяю год в дате {str_d} : с {year_true} на {year_false}')
#                 return pd.to_datetime(str_d.replace(year_false, year_true), format="%d.%m.%Y")
        
#         return pd.to_datetime(str_d, format="%d.%m.%Y")
    
#     except:
#         assert 0, f'Ошибка, строка = "{str_d}" не конвертируется'

        
# df_concat_ed = df_final.copy()


# for n_date in ['Дата возбуждения', 'Дата выписки исполнительного документа']:
#     print('   ', n_date)
#     df_concat_ed[n_date] = df_concat_ed[n_date].apply(to_date)
#     print('+ Все даты верные')
    
# #df_concat_ed.drop('date_end', axis=1, inplace=True)


# ### Убираю дубли

# In[ ]:


# df_final_ed = df_concat_ed.copy()   # Файл после преоразования даты!!!
# # df_final_ed['Дата возбуждения'] = pd.to_datetime(df_final_ed['Дата возбуждения'], format='%d.%m.%Y')
# # df_final_ed = df_final_ed.sort_values(by=['Дата возбуждения'], ascending=False).drop_duplicates('ИИН/БИН должника', keep='first')
# # # df_final_ed.to_excel(f'{loading_path}/{name_login} AIS_OIP {now_str} Итог v2.xlsx', index=False)


# # Подключение google sheets

# ## Получаю данные из БД

# In[ ]:


import string
from pprint import pprint
import pandas as pd
import gspread
from gspread import Cell, Client, Spreadsheet, Worksheet
from gspread.utils import rowcol_to_a1
import requests


SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/1DBZnYTpO5vaTh81aELCUuO2DGUSzRSWRHMpPLK8qNg0/edit?usp=drivesdk'



def show_available_worksheets(sh: Spreadsheet):
    worksheets = sh.worksheets()

    for ws in worksheets:
        print("Worksheet with title", repr(ws.title), "and id", ws.id)


def get_dataframe_from_sheet(sh: Spreadsheet, sheet_title: str) -> pd.DataFrame:
    # Получаем рабочий лист по названию
    worksheet = sh.worksheet(sheet_title)
    
    # Получаем все данные из листа
    data = worksheet.get_all_values()
    
    # Создаем DataFrame из полученных данных
    # Первая строка используется в качестве заголовков столбцов
    df = pd.DataFrame(data[1:], columns=data[0])
    
    return df
        

def get_google_df():
    gc: Client = gspread.service_account("./credentials kpi-api.json")
    sh: Spreadsheet = gc.open_by_url(SPREADSHEET_URL)

    show_available_worksheets(sh)    
    df = get_dataframe_from_sheet(sh, "ММ")
    return df
    
df_google_t = get_google_df()


# In[ ]:


# pd.set_option('display.max_columns', None)
# df_google_t.head(5)


# In[ ]:


# df_google_t.to_excel("df_google_t.xlsx")


# In[ ]:





# ### Редактирую данные из бд.

# In[ ]:


def to_int(v):
    v = str(v)
    if len(v.split('.')) > 2 :
        raise f'Ошибка строка {v}'
    
    v = v.replace(' ', '')
    v = v.replace('-', '')
    v = v.replace(',', '.')
    v = v.replace('\xa0', '')
    v = v.split('.')[0]
    if len(v) == 0:
        return 0
    else:
        return int(v)


df_google_t['ОД_claen'] = df_google_t['ОД'].apply(to_int)


columns = ['Сумма выдачи', 'ОД', 'Вознаграждение', 'Пеня', 'Всего задолженность при покупке', 'нотариальные услуги', 'сумма оплаты после цессии', 'остаток задолженности', 'Сумма оплат', 'госпошина по ауэз суду', 'госпошина по мед суду']
for col in columns:
    print('col', col)
    df_google_t[col] = df_google_t[col].apply(to_int)
# df = df['Всего задолженность при покупке']


# In[ ]:


df_google_t['госпошина по медеу суду'] = df_google_t['госпошина по мед суду']
df_google_t['ИИН'] = df_google_t['ИИН'].apply(lambda x: '0' * (12 - len(str(x))) + str(x) )


# Добавить колонки.

# In[ ]:


# Остаток задолжности
# Тотал по покупке + нат надпись. 

# адк названия. (Вроде так. Возможны расхождения.)
# Value # ОД 
# f140   # Тотал (за вычетом погашений)
# f148     # Тотал (по покупке) # Всего задолженность при покупке
                  
# f114,    # Тотал (на момент передачи нотариусу)
# f118,    # Тотал (на момент иска в суд)
# f147,    # Тотал (по решению суда)
# f386    # Тотал (по медиации/мировому соглашению)


# omega названия

# 1) f1101 - Всего задолженность при покупке
# 2) f1102 - Всего задолженность при покупке + нотариальные услуги
# 3) f1103 - Всего задолженность при покупке + госпошина по суду
# 4) f1104 - Всего задолженность при покупке + госпошина по суду + нотариальные услуги

# 5) f1105 - остаток задолженности
# 6) f1106 - остаток задолженности + нотариальные услуги
# 7) f1107 - остаток задолженности + госпошина по суду
# 8) f1108 - остаток задолженности + госпошина по суду + нотариальные услуги

# 9) f1109 - сумма АИСОИП.
df_google_t['госпошина по суду'] = df_google_t['госпошина по ауэз суду'] + df_google_t['госпошина по медеу суду']

df_google_t['f1101'] = df_google_t['Всего задолженность при покупке']
df_google_t['f1102'] = df_google_t['Всего задолженность при покупке'] + df_google_t['нотариальные услуги']
df_google_t['f1103'] = df_google_t['Всего задолженность при покупке'] + df_google_t['госпошина по суду']
df_google_t['f1104'] = df_google_t['Всего задолженность при покупке'] + df_google_t['нотариальные услуги'] + df_google_t['нотариальные услуги']

df_google_t['f1105'] = df_google_t['остаток задолженности']
df_google_t['f1106'] = df_google_t['остаток задолженности'] + df_google_t['нотариальные услуги']
df_google_t['f1107'] = df_google_t['остаток задолженности'] + df_google_t['госпошина по суду']
df_google_t['f1108'] = df_google_t['остаток задолженности'] + df_google_t['нотариальные услуги'] + df_google_t['нотариальные услуги']

df_google_t['f1109'] = df_google_t['Сумма по АИСОИП']

df_google_t['f1110'] = df_google_t['остаток задолженности'] - 685


# In[ ]:


# df_google_t['f1101'] = df_google_t['Всего задолженность при покупке']
# df_google_t['f1102'] = df_google_t['Всего задолженность при покупке'] + df_google_t['нотариальные услуги']
# df_google_t['f1103'] = df_google_t['Всего задолженность при покупке'] + df_google_t['госпошина по суду']
# df_google_t['f1104'] = df_google_t['Всего задолженность при покупке'] + df_google_t['нотариальные услуги'] + df_google_t['нотариальные услуги']

# df_google_t['f1105'] = df_google_t['остаток задолженности']
# df_google_t['f1106'] = df_google_t['остаток задолженности'] + df_google_t['нотариальные услуги']
# df_google_t['f1107'] = df_google_t['остаток задолженности'] + df_google_t['госпошина по суду']
# df_google_t['f1108'] = df_google_t['остаток задолженности'] + df_google_t['нотариальные услуги'] + df_google_t['нотариальные услуги']

# df_google_t['f1109'] = df_google_t['Сумма по АИСОИП']


# In[ ]:


recode_old_new = [('ИИН', 'inn'), 
('index', 'eid'),
('index', 'id'), 
('f1101', 'f1101'),
('f1102', 'f1102'),
('f1103', 'f1103'),
('f1104', 'f1104'),
('f1105', 'f1105'),
('f1106', 'f1106'),
('f1107', 'f1107'),
('f1108', 'f1108'),
('f1109', 'f1109'),
('f1110', 'f1110'),
                  
('Номер договора цессии', 'f258'), # Это не правильное поле !!! TODO. Надо добавить в место него "Номер испол. производ" из АИСОИП                
# ('ОД', 'Value'),  # 'ОД'
# ('остаток задолженности', 'f140'),   # Тотал (за вычетом погашений)
# ('f148_v1', 'f148'),     # Тотал (по покупке) # Всего задолженность при покупке
                  
# ('ОД_claen', 'f114'),    # Тотал (на момент передачи нотариусу)
# ('ОД_claen', 'f118'),    # Тотал (на момент иска в суд)
# ('ОД_claen', 'f147'),    # Тотал (по решению суда)
# ('ОД_claen', 'f386'),    # Тотал (по медиации/мировому соглашению)
]

df_google_t['index'] = df_google_t.index


# In[ ]:


# df.to_excel('df db.xlsx')


# In[ ]:


df_db_clients = pd.DataFrame() # df[['eid', ]] #.head()
for old, new in recode_old_new:
    addition_col = df_google_t[old]
    df_db_clients[new] = addition_col
    print('old, new', old, new)


# # Скачиваю данные AISOIP

# In[ ]:


# df_ais = pd.read_excel('Взыскатель-20240225-21-49.xlsx')


df_ais = df_final.copy()


# In[ ]:


df_ais['ИИН/БИН должника'] = df_ais['ИИН/БИН должника'].apply(lambda x: '0' * (12 - len(str(x))) + str(x) )


# In[ ]:


ais_col = ['№',
'Номер исполнительного документа', 
'Номер исполнительного производства', 
'Судебный исполнитель', 
'Дата возбуждения', 
'Дата выписки исполнительного документа', 
'Орган выдавший исполнительный документ', 
'Тип взыскания',
'Должник',
'ИИН/БИН должника', 
'Статус исполнительного производства', 
'Сумма']

df_ais_oip_raw_data = df_ais[ais_col]


# ### Загрузка данных в локальную БД SQLite

# In[ ]:


import sqlite3
import pandas as pd

# Предположим, что у вас уже есть DataFrame под названием df

# Создание соединения с базой данных SQLite
conn = sqlite3.connect('my_database.sqlite')

# Загрузка данных из DataFrame в таблицу SQLite
df_ais_oip_raw_data.to_sql('AKA_ais_oip_raw_data', conn, if_exists='replace', index=False)
df_db_clients.to_sql('db_clients', conn, if_exists='replace', index=False)

# Закрытие соединения с базой данных
conn.close()


# ### Запрос на SQLite

# In[ ]:


sql = '''
with doubles as  
(
    select t.*
         , ROW_NUMBER() over(partition by "Номер исполнительного документа",
                                          "Номер исполнительного производства",
                                          "Судебный исполнитель",
                                          "ИИН/БИН должника",
                                          "Статус исполнительного производства",
                                          "Сумма",
                                          "Дата возбуждения"
                             order by "Дата возбуждения" desc
                          ) as rn
      from AKA_ais_oip_raw_data t
),
no_doubles as 
(
    select *
      from doubles
     where rn = 1 
),
clients as 
(
    select id, inn, f258, eid
         , IFNULL(round(f1101, 0), 0) as f1101
         , IFNULL(round(f1102, 0), 0) as f1102
         , IFNULL(round(f1103, 0), 0) as f1103
         , IFNULL(round(f1104, 0), 0) as f1104
         , IFNULL(round(f1105, 0), 0) as f1105
         , IFNULL(round(f1106, 0), 0) as f1106
         , IFNULL(round(f1107, 0), 0) as f1107
         , IFNULL(round(f1108, 0), 0) as f1108
         , IFNULL(round(f1109, 0), 0) as f1109
         , IFNULL(round(f1110, 0), 0) as f1110
      from db_clients c
      -- Примечание: Убедитесь, что соединения таблиц и условия соответствуют вашему запросу
      -- Этот блок может потребовать дополнительной адаптации в соответствии с вашей схемой БД
),
right_data as 
(
    select d."№"
         , d."Номер исполнительного документа"    
         , d."Номер исполнительного производства"    
         , d."Судебный исполнитель"
         , d."Дата возбуждения"
         , d."Дата выписки исполнительного документа"
         , d."Орган выдавший исполнительный документ"
         , d."Тип взыскания"
         , d."Должник"
         , d."ИИН/БИН должника"
         , d."Статус исполнительного производства"
         , d."Сумма"
         , c.eid as "Уникальный идентификатор займа"
      from no_doubles d
      join clients c
        on c.inn = d."ИИН/БИН должника"
       and c.f258 = d."Номер исполнительного производства"
),



wrong_data as 
(
    select d."№"
         , d."Номер исполнительного документа"    
         , d."Номер исполнительного производства"    
         , d."Судебный исполнитель"
         , d."Дата возбуждения"
         , d."Дата выписки исполнительного документа"
         , d."Орган выдавший исполнительный документ"
         , d."Тип взыскания"
         , d."Должник"
         , d."ИИН/БИН должника"
         , d."Статус исполнительного производства"
         , d."Сумма"
      from no_doubles d
    except
    select r."№"
         , r."Номер исполнительного документа"    
         , r."Номер исполнительного производства"    
         , r."Судебный исполнитель"
         , r."Дата возбуждения"
         , r."Дата выписки исполнительного документа"
         , r."Орган выдавший исполнительный документ"
         , r."Тип взыскания"
         , r."Должник"
         , r."ИИН/БИН должника"
         , r."Статус исполнительного производства"
         , r."Сумма"
      from right_data r
),
right_data2 as 
(
    select d.*, c.eid as "Уникальный идентификатор займа"
      from wrong_data d
      join clients c
        on c.inn = d."ИИН/БИН должника"
       and (
            (d."Сумма" between c.f1101-3 and c.f1101+3 and c.f1101 != 0)
            or (d."Сумма" between c.f1102-3 and c.f1102+3 and c.f1102 != 0)
            or (d."Сумма" between c.f1103-3 and c.f1103+3 and c.f1103 != 0)
            or (d."Сумма" between c.f1104-3 and c.f1104+3 and c.f1104 != 0)
            or (d."Сумма" between c.f1105-3 and c.f1105+3 and c.f1105 != 0)
            or (d."Сумма" between c.f1106-3 and c.f1106+3 and c.f1106 != 0)
            or (d."Сумма" between c.f1107-3 and c.f1107+3 and c.f1107 != 0)
            or (d."Сумма" between c.f1108-3 and c.f1108+3 and c.f1108 != 0)
            or (d."Сумма" between c.f1109-3 and c.f1109+3 and c.f1109 != 0)
            or (d."Сумма" between c.f1110-3 and c.f1110+3 and c.f1110 != 0)

           )
),
wrong2 as 
(
    select d.*, null as eid
      from wrong_data d
      left join clients c
        on c.inn = d."ИИН/БИН должника"
       and (
            (d."Сумма" between c.f1101-3 and c.f1101+3 and c.f1101 != 0)
            or (d."Сумма" between c.f1102-3 and c.f1102+3 and c.f1102 != 0)
            or (d."Сумма" between c.f1103-3 and c.f1103+3 and c.f1103 != 0)
            or (d."Сумма" between c.f1104-3 and c.f1104+3 and c.f1104 != 0)
            or (d."Сумма" between c.f1105-3 and c.f1105+3 and c.f1105 != 0)
            or (d."Сумма" between c.f1106-3 and c.f1106+3 and c.f1106 != 0)
            or (d."Сумма" between c.f1107-3 and c.f1107+3 and c.f1107 != 0)
            or (d."Сумма" between c.f1108-3 and c.f1108+3 and c.f1108 != 0)
            or (d."Сумма" between c.f1109-3 and c.f1109+3 and c.f1109 != 0)
            or (d."Сумма" between c.f1110-3 and c.f1110+3 and c.f1110 != 0)
           )
      where c.id is null
),
right_data3 as 
(
    select d."№"
         , d."Номер исполнительного документа"    
         , d."Номер исполнительного производства"    
         , d."Судебный исполнитель"
         , d."Дата возбуждения"
         , d."Дата выписки исполнительного документа"
         , d."Орган выдавший исполнительный документ"
         , d."Тип взыскания"
         , d."Должник"
         , d."ИИН/БИН должника"
         , d."Статус исполнительного производства"
         , d."Сумма" 
         , c.eid as eid
      from wrong2 d
      join (
            select c.inn, count(*) as cnt_iin, max(eid) as eid
            from clients c
            group by c.inn
           ) c
        on c.inn = d."ИИН/БИН должника"
       and c.cnt_iin = 1
),


wrong3 as 
(
    select *
    from wrong2 w2
    where not exists (
        select 1 from right_data3 rd3 where rd3."ИИН/БИН должника" = w2."ИИН/БИН должника"
    )
),


fin as 
(
    select *, 
           case 
               when "Статус исполнительного производства" = 'На исполнении' then 1 
               else 2 
           end as priority_id_status
      from right_data
    union all
    select *, 
           case 
               when "Статус исполнительного производства" = 'На исполнении' then 1 
               else 2 
           end as priority_id_status
      from right_data2
    union all
    select *, 
           case 
               when "Статус исполнительного производства" = 'На исполнении' then 1 
               else 2 
           end as priority_id_status
      from right_data3
),


fin2 as
(
    select *, 
           ROW_NUMBER() over(
               partition by "Уникальный идентификатор займа"
               order by priority_id_status, "Дата возбуждения" desc, "Дата выписки исполнительного документа" desc, "Статус исполнительного производства"
           ) as rn
      from fin
)
select "№",
       "Номер исполнительного документа",    
       "Номер исполнительного производства",    
       "Судебный исполнитель",
       strftime('%d.%m.%Y', "Дата возбуждения") as "Дата возбуждения",
       strftime('%d.%m.%Y', "Дата выписки исполнительного документа") as "Дата выписки исполнительного документа",
       "Орган выдавший исполнительный документ",
       "Тип взыскания",
       "Должник",
       "ИИН/БИН должника",
       "Статус исполнительного производства",
       "Сумма",
       "Уникальный идентификатор займа"
from fin2 
where rn = 1
union
select "№",
       "Номер исполнительного документа",    
       "Номер исполнительного производства",    
       "Судебный исполнитель",
       strftime('%d.%m.%Y', "Дата возбуждения") as "Дата возбуждения",
       strftime('%d.%m.%Y', "Дата выписки исполнительного документа") as "Дата выписки исполнительного документа",
       "Орган выдавший исполнительный документ",
       "Тип взыскания",
       "Должник",
       "ИИН/БИН должника",
       "Статус исполнительного производства",
       "Сумма",
       null
from wrong3
'''


# In[ ]:


# Открытие соединения с базой данных SQLite
conn = sqlite3.connect('my_database.sqlite')

# Пример: выбрать все данные из таблицы db_clients
query = sql
df_sql = pd.read_sql_query(query, conn)

# Закрытие соединения с базой данных
conn.close()


# In[ ]:


df_sql['Уникальный идентификатор займа'] = df_sql['Уникальный идентификатор займа'].astype('Int64')


# In[ ]:


# Проверки


col = '№'
cnt_load_check = df_sql.groupby(['Уникальный идентификатор займа'], as_index=False).agg({col:'count'}).rename(columns={col:'cnt_load'})
cnt_duble = (cnt_load_check['cnt_load'] > 1).sum()

print('Кол-во дублей', cnt_duble)
print('Кол-во пустых значений', df_sql['Уникальный идентификатор займа'].isna().sum())

is_finish = (df_google_t['статус кредита'] == 'погашен') | (df_google_t['статус кредита'] == 'обратный выкуп')

df_google_t.loc[~is_finish, 'index'].isin(df_sql['Уникальный идентификатор займа'])
cnt_not_pull = (~df_google_t.loc[~is_finish, 'index'].isin(df_sql['Уникальный идентификатор займа'])).sum()

print('Кол-во дел по которым не подтянулось (из незавершонных)', cnt_not_pull)


# In[ ]:


# df_google_t = df_google_t.drop("Номер_строки_google_t", axis=1)
# df_google_t


# In[ ]:


# Подтягиваю под исходный формат
df_google_t.rename(columns={'index':'Номер_строки_google_t'}, inplace=True)
df_google_t['Номер_строки_google_t'] = df_google_t['Номер_строки_google_t'].astype('Int64')

df_google_t['Номер_строки_google_t'] = df_google_t['Номер_строки_google_t'].astype('Int64')
final_table = df_google_t[['Номер_строки_google_t', 'ИИН']].merge(df_sql, how='left', left_on=['Номер_строки_google_t'], right_on=['Уникальный идентификатор займа'])


# In[ ]:


final_table.sort_values('Номер_строки_google_t', inplace=True)


# In[ ]:


def uid_to_string(uid, first_num=2, cnt_num=6):
    """
    uid = unique_id
    suid = string_uid

    input 1
    output 100_001

    input 11
    output 100_011

    """

    suid = str(uid)
    cnt_zero =  cnt_num - 1 - len(suid)
    new_suid = str(first_num) + "0" * cnt_zero + suid
    return new_suid


final_table['уникальный номер'] = final_table['Номер_строки_google_t'] + 1     # Старт не с "0" а с "1".
final_table['уникальный номер'] = final_table['уникальный номер'].apply(uid_to_string)
final_table = final_table.drop('Уникальный идентификатор займа', axis=1)


# In[ ]:


now_str = datetime.now().strftime("%d.%m.%Y")
final_table.to_excel(f'{loading_path}/{name_login} AIS_OIP_merge_google_table {now_str}.xlsx', index=False)


# In[ ]:


df_ais_not_processed = df_ais_oip_raw_data.loc[(~df_ais_oip_raw_data['Номер исполнительного производства'].isin(df_sql["Номер исполнительного производства"])) & (df_ais_oip_raw_data['Статус исполнительного производства'] == 'На исполнении')]
df_ais_not_processed.to_excel(f'{loading_path}/{name_login} Производства не подтянулись .xlsx', index=False)

print(f'Производств в работе не найдено: {len(df_ais_not_processed)}')


# # Убираю из проверенных отмененных (не отменённые статусы)

# In[ ]:


def clean_cancel_list(new_clean_df, path_load_finish):
    '''
    Очитаю эскль в котором хранятся данные по отменам которыне не нужно проверять. 
    '''
    new_clean_df['Дата возбуждения'] = pd.to_datetime(new_clean_df['Дата возбуждения'], format='%d.%m.%Y')
    new_clean_df = new_clean_df.sort_values(by=['Дата возбуждения'], ascending=False).drop_duplicates('ИИН/БИН должника', keep='first')


    path_check_iin = Path(path_load_finish).joinpath('Проверенные ранее.xlsx')
    df_old_ = pd.read_excel(path_check_iin, dtype='str')
    
    df_old_['ИИН_and_id_Испол'] = df_old_['ИИН'] + '_' + df_old_['Номер исполнительного производства']
    new_clean_df['ИИН_and_id_Испол'] = new_clean_df['ИИН'] + '_' + new_clean_df['Номер исполнительного производства']
    

    df_drop = new_clean_df.loc[new_clean_df['Статус исполнительного производства'] != 'Окончено', 'ИИН_and_id_Испол']
    df_new_check = df_old_[~df_old_['ИИН_and_id_Испол'].isin(df_drop)].copy()
    df_new_check.to_excel(path_check_iin, index=False)

    print(f'Удалено из проверенных {df_old_.shape[0] - df_new_check.shape[0]}')


clean_cancel_list(final_table, path_load_finish)


# # Создание дополнительного файла с форматами

# ## Поиск отмен

# In[ ]:


# import os
import re
# from datetime import datetime
import glob

def get_last_cancel_path(load_path):
    # Регулярное выражение для поиска даты в названии папок
    date_pattern = re.compile(r"(\d{4}-\d{2}-\d{2})___\d{2}-\d{2}")

    latest_date = None
    latest_folder = None

    for folder in os.listdir(load_path):
        folder_path = os.path.join(load_path, folder)

        if not os.path.isdir(folder_path):  # Пропускаем файлы
            continue

        match = date_pattern.match(folder)
        if match:
            folder_date = datetime.strptime(match.group(1), "%Y-%m-%d")
            if latest_date is None or folder_date > latest_date:
                latest_date = folder_date
                latest_folder = folder

    print(f"Самая последняя папка: {latest_folder}")
    return latest_folder

latest_folder_cancel = get_last_cancel_path(path_load_finish)


# In[ ]:


# Полный путь к последней папке
latest_folder_path = os.path.join(path_load_finish, latest_folder_cancel)

# Ищем файл с "Итог_union" в названии
file_pattern = os.path.join(latest_folder_path, "*Итог_union*")
matching_files = glob.glob(file_pattern)

# Выбираем первый найденный файл (если есть)
if matching_files:
    target_file = matching_files[0]
    print(f"Найденный файл: {target_file}")
    df_last_cancel = pd.read_excel(target_file)
else:
    print("Файл с 'Итог_union' не найден.")


# ## Отмены + проверенные ранее

# In[ ]:


check_later = os.path.join(path_load_finish, 'Проверенные ранее.xlsx')
df_check_later = pd.read_excel(check_later)


# In[ ]:


df_last_cancel = df_last_cancel.rename(columns={"inn":'ИИН', 
                                                "f258":'Номер исполнительного производства', 
                                                "Класс 1":"Класс 1",
                                                "Дата 1":"Дата 1"
                                               })


# In[ ]:


df_check_later["Дата 1"] = pd.to_datetime(df_check_later["Дата 1"])
df_last_cancel["Дата 1"] = pd.to_datetime(df_last_cancel["Дата 1"], format="%d.%m.%Y")


# In[ ]:


# df_cancel = pd.concat([df_last_cancel[['ИИН', 'Номер исполнительного производства', 'Класс 1']], df_check_later[['ИИН', 'Номер исполнительного производства', 'Класс 1'
# ]]])

df_cancel = pd.concat([
    df_check_later[['ИИН', 'Номер исполнительного производства', 'Класс 1', "Дата 1"]], 
    df_last_cancel[['ИИН', 'Номер исполнительного производства', 'Класс 1', "Дата 1"]] ])

df_cancel = df_cancel.dropna()
mask = (df_cancel["Класс 1"].notna()) & (df_cancel["Класс 1"] != "")
df_cancel = df_cancel.loc[mask]


# In[ ]:


df_cancel = df_cancel.rename(columns={"Класс 1":"Статус исполнительного документа", "Дата 1":"Дата Постановления о прекращении ИП"})
df_cancel  = df_cancel.drop_duplicates(["ИИН", "Номер исполнительного производства"])


# ## Создание нового файла.

# In[ ]:


col_target = ["Уникальный номер сделки", "Статус АИС ОИП", "ЧСИ", "Дата возбуждения судебного производства", "Сумма иска", "Номер исполнительного производства", "Номер исполнительного документа", "Статус исполнительного документа", "Дата выписки исполнительного документа", "Орган выдавший исполнительный документ", "Дата Постановления о прекращении ИП"]


# In[ ]:


final_table_col_rename = final_table.copy()


# In[ ]:


rename_columns = {
"Уникальный идентификатор займа":"Уникальный номер сделки",
"Статус исполнительного производства":"Статус АИС ОИП",
"Судебный исполнитель":"ЧСИ",
"Дата возбуждения":"Дата возбуждения судебного производства",
"Сумма":"Сумма иска",
"Номер исполнительного производства":"Номер исполнительного производства",
"Номер исполнительного документа":"Номер исполнительного документа",
111:"Статус исполнительного документа",
"Дата выписки исполнительного документа":"Дата выписки исполнительного документа",
"Орган выдавший исполнительный документ":"Орган выдавший исполнительный документ"
}

final_table_col_rename = final_table.rename(columns=rename_columns)
# final_table_col_rename["Уникальный номер сделки"] = final_table_col_rename["Уникальный номер сделки"] + 100000
final_table_col_rename["Уникальный номер сделки"] = final_table_col_rename['уникальный номер'] # + 100000


# In[ ]:


# 
final_table_col_rename["ИИН"] = final_table_col_rename["ИИН"].astype(np.int64)
df_cancel["ИИН"] = df_cancel["ИИН"].astype(np.int64)


# In[ ]:


final_table_col_rename = pd.merge(final_table_col_rename, df_cancel, on=["ИИН", "Номер исполнительного производства"], how='left')


# In[ ]:


final_table_col_rename = final_table_col_rename[col_target]



# In[ ]:


now_str = datetime.now().strftime("%d.%m.%Y")
path_format1 = f'{loading_path}/AIS_OIP_Import.xlsx'

final_table_col_rename.to_excel(path_format1, index=False)



# In[ ]:


template_path = f'{loading_path_part}/Шаблон1.xlsx'


# In[ ]:


final_table_col_rename.to_excel(path_format1, index=False)



import win32com.client as win32

def apply_format_and_header_from_template(template_path, target_path):
    # Инициализируем Excel
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    excel.Visible = False  # Отключаем видимость

    try:
        # Открываем файлы
        template_wb = excel.Workbooks.Open(template_path)
        target_wb = excel.Workbooks.Open(target_path)

        # Выбираем первый лист в обоих файлах
        template_ws = template_wb.Sheets(1)
        target_ws = target_wb.Sheets(1)

        # 1. Копируем формат заголовка (первой строки) из шаблона в целевой файл
        template_header_range = template_ws.Rows(1)  # Заголовок в шаблоне
        target_header_range = target_ws.Rows(1)      # Заголовок в целевом файле
        template_header_range.Copy()
        target_header_range.PasteSpecial(Paste=-4104)  # -4163 = xlPasteFormats

        # 2. Применяем формат второй строки из шаблона ко всем строкам целевого файла
        template_second_row_format = template_ws.Rows(2)  # Вторая строка в шаблоне
        target_last_row = target_ws.UsedRange.Rows.Count  # Последняя используемая строка в целевом файле
        target_body_range = target_ws.Rows(f"2:{target_last_row}")  # Диапазон строк, начиная со второй
        template_second_row_format.Copy()
        target_body_range.PasteSpecial(Paste=-4122)  # Применяем только форматирование

        # 3. Копируем ширину столбцов из шаблона в целевой файл
        template_used_columns = template_ws.UsedRange.Columns.Count  # Количество столбцов в шаблоне
        for col in range(1, template_used_columns + 1):
            template_column_width = template_ws.Columns(col).ColumnWidth  # Ширина столбца в шаблоне
            target_ws.Columns(col).ColumnWidth = template_column_width      # Применяем ширину в целевом файле

        # Сохраняем изменения в целевом файле
        target_wb.Save()
    finally:
        # Закрываем файлы и Excel
        template_wb.Close(SaveChanges=False)
        target_wb.Close(SaveChanges=True)
        excel.Quit()

# # Укажите пути к файлу шаблона и целевому файлу

apply_format_and_header_from_template(template_path, path_format1)


# # копирование файла в папку crm

# In[ ]:


path_format1 = Path(path_format1)
path_crm = Path(path_crm)

# Формирование полного пути для целевого файла
path_crm_final = path_crm / path_format1.name


# In[ ]:


try:
    # Копирование файла, включая метаданные
    shutil.copy2(path_format1, path_crm_final)
    print(f"Файл успешно скопирован в: {path_crm_final}")
except Exception as e:
    print(f"Ошибка при копировании файла: {e}")


# In[ ]:





# # 

# # 


*** kpi ***
date and time = 2025-12-10___17-44
try =  1
2025-12-10
1-5 из 38002
try =  2
2025-12-10
Слишком долгая загрузка сайта. Обновляю странцу.
invalid literal for int() with base 10: '–'
 Error occurred in file C:\Users\User\AppData\Local\Temp\ipykernel_13624\3813591460.py, line 424: cnt_iter, cnt_row_text = find_cnt_all_row(driver)
try =  3
Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7d0148255
	0x7ff7d01482b0
	0x7ff7cff2165d
	0x7ff7cff79a33
	0x7ff7cff79d3c
	0x7ff7cffcdf67
	0x7ff7cffcac97
	0x7ff7cff6ac29
	0x7ff7cff6ba93
	0x7ff7d045fff0
	0x7ff7d045a930
	0x7ff7d0479096
	0x7ff7d0165754
	0x7ff7d016e6fc
	0x7ff7d0151974
	0x7ff7d0151b25
	0x7ff7d0137852
	0x7ff8a466e8d7
	0x7ff8a68ac53c

 Error occurred in file C:\Users\User\AppData\Local\Temp\ipykernel_13624\3813591460.py, line 344: write_login(driver, name_login)
try =  4
Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7d0148255
	0x7ff7d01482b0
	0x7ff7cff2165d
	0x7ff7c

KeyboardInterrupt: 